In [2]:
from matplotlib import pyplot as plt
import zipfile
import tempfile
import requests
import os
import numpy as np
import pandas as pd

In [158]:
#GLOBAL
SELECT_ALL_QUESTIONS = ['SPNDSRC1', 'SPNDSRC2', 'SPNDSRC3', 'SPNDSRC4', 'SPNDSRC5', 'SPNDSRC6', 'SPNDSRC7', 'SPNDSRC8']

def get_puf_data(first_week, last_week):
    '''
    download puf files for the given weeks and concatenate the datasets
    '''
    df = pd.DataFrame()
    for i in range(first_week,last_week+1):
        y = 2020
        if i>=22:
            y=2021
        file_str = "pulse{}_puf_{}.csv".format(y,i)
        url_str = "https://www2.census.gov/programs-surveys/demo/datasets/hhp/{yr}/wk{w}/HPS_Week{w}_PUF_CSV.zip".format(yr=y,w=i)
        week_df = download_url(url_str, file_str, y, i)
        df = pd.concat([df,week_df])

    return df


def download_url(url_str,file_str, year, week, chunk_size=128):
    '''
    download puf, avoid intermediate save to disk
    '''
    df = pd.DataFrame()
    r = requests.get(url_str)
    print(r)
    #temp directory rather than file to hopefully accomodate windows users
    with tempfile.TemporaryDirectory() as td:
        f_name = os.path.join(td, 'week_{}'.format(week))
        with open(f_name, 'wb') as fh:
            for chunk in r.iter_content(chunk_size=chunk_size):
                fh.write(chunk)
        zf = zipfile.ZipFile(f_name)
        df = pd.read_csv(zf.open(file_str))
        df1 = pd.read_csv(zf.open("pulse{}_repwgt_puf_{}.csv".format(year,week)))
        zf.close()
        r.close()

    return pd.merge(df,df1,on=["SCRAM","WEEK"],how="inner")


def get_std_err(df, weight):
    #make 1d array of weight col
    obs_wgts = df[weight].to_numpy().reshape(len(df),1)
    
    #make 80d array of replicate weights
    rep_wgts = df[[i for i in df.columns if weight in i and not i == weight]].to_numpy()
    
    #return standard error of estimate
    return np.sqrt((np.sum(np.square(rep_wgts-obs_wgts),axis=1)*(4/80)))
############

def freq_crosstab(df, col_list, weight, critical_val=1):
    '''

    '''
    
    pt_estimates = df.groupby(col_list, as_index=True)[[i for i in df.columns if weight in i]].agg('sum')
    pt_estimates['std_err'] = get_std_err(pt_estimates, weight)
    pt_estimates['mrgn_err'] = pt_estimates.std_err * critical_val

    return pt_estimates[[weight, 'std_err','mrgn_err']].reset_index()

def full_crosstab(df, col_list, weight, proportion_level, critical_val=1):
    df1 = df.copy()
    detail = freq_crosstab(df1, col_list, weight, critical_val)
    top = freq_crosstab(df1, proportion_level, weight, critical_val)
    
    rv = detail.merge(top,'left',proportion_level,suffixes=('_full','_demo'))
    rv['proportions'] = rv[weight+'_full']/rv[weight+'_demo']
    return rv

def bulk_crosstabs(df, idx_list, ct_list, q_list, weight='PWEIGHT', critical_val=1):
    rv = pd.DataFrame()
    input_df = df.copy()
    input_df[SELECT_ALL_QUESTIONS] = input_df[SELECT_ALL_QUESTIONS].replace('-99','0 - not selected')
    input_df.replace(['-88','-99'],np.nan,inplace=True)
    for ct in ct_list:
        for q in q_list:
            full = idx_list + [ct, q]
            abstract = idx_list + [ct]
            temp = input_df[-input_df[full].isna().any(axis=1)]
            curr_bin = full_crosstab(temp,full,
                            weight,
                            abstract,
                            critical_val=critical_val).round(2)
            curr_bin.rename(columns={q:'q_val',ct:'ct_val'},inplace=True)
            curr_bin['ct_var'] = ct
            curr_bin['q_var'] = q
            rv = pd.concat([rv,curr_bin])
    return rv

In [87]:
core_df = get_puf_data(13,22)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


## Tie Out Housing 1b Table

In [90]:
test_df = core_df[core_df.WEEK==21].copy()
test_df['rent_check'] = test_df.apply(lambda x: 4 if x.TENURE==4 else x.RENTCUR, axis=1)
test_df['race_eth'] = test_df.apply(lambda x: 5 if x.RHISPANIC==2 else x.RRACE, axis=1)
full_crosstab(test_df, ['EGENDER', 'rent_check'], 'PWEIGHT', ['EGENDER'],critical_val=1).round(2)

,EGENDER,rent_check,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions
0,1,-99,65521.28,22916.93,22916.93,120531610.0,0.0,0.0,0.00
1,1,-88,95767609.18,616700.77,616700.77,120531610.0,0.0,0.0,0.79
2,1,1,18984062.29,625926.33,625926.33,120531610.0,0.0,0.0,0.16
3,1,2,4161891.23,265570.79,265570.79,120531610.0,0.0,0.0,0.03
4,1,4,1552526.03,178942.97,178942.97,120531610.0,0.0,0.0,0.01
5,2,-99,52740.11,18792.06,18792.06,128639306.0,0.0,0.0,0.00
6,2,-88,97179048.55,516318.86,516318.86,128639306.0,0.0,0.0,0.76
7,2,1,23721816.46,458667.57,458667.57,128639306.0,0.0,0.0,0.18
8,2,2,5974193.24,289729.34,289729.34,128639306.0,0.0,0.0,0.05
9,2,4,1711507.64,145060.18,145060.18,128639306.0,0.0,0.0,0.01


In [44]:
states = {1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',
    8:'Colorado',9:'Connecticut',10:'Delaware',11:'District of Columbia',12:'Florida',
    13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',21:'Kentucky',
    22:'Louisiana',23:'Maine',24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota',
    28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',
    34:'New Jersey',35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',
    40:'Oklahoma',41:'Oregon',42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',
    46:'South Dakota',47:'Tennessee',48:'Texas',49:'Utah',50:'Vermont',51:'Virginia',
    53:'Washington',54:'West Virginia',55:'Wisconsin',56:'Wyoming'}
regions = {1:'NE',2:'S',3:'MW',4:'W'}
metros = {35620:'New York',31080:'Los Angeles',16980:'Chicago',19100:'Dallas',
          26420:'Houston',47900:'DC',33100:'Miami',37980:'Philadelphia',
          12060:'Atlanta',38060:'Phoenix',14460:'Boston',41860:'San Francisco',
          40140:'Riverside CA',19820:'Detroit',42660:'Seattle'}
races = {1:'white',2:'black',3:'Asian',4:'Other'}
rents = {1:'current',2:'behind'}
df1 = core_df.copy()
df1['metro'] = df1.EST_MSA.map(metros) 
df1['state'] = df1.EST_ST.map(states)
df1['region'] = df1.REGION.map(regions)
df1['race'] = df1.RRACE.map(races)
df1['rent'] = df1.RENTCUR.map(rents)

In [17]:
df1.head()

,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,...,EIPSPND4,EIPSPND5,EIPSPND6,EIPSPND7,EIPSPND8,EIPSPND9,EIPSPND10,EIPSPND11,EIPSPND12,EIPSPND13
0,V130000001S12010403200123,13,48,19100.0,2,1276.886650,1278.107099,1989,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V130000001S34010154300113,13,1,NaN,2,1195.578846,2323.078960,1988,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V130000001S37010241600123,13,1,NaN,2,1124.648326,1092.628425,1969,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V130000001S37010645600123,13,1,NaN,2,517.598058,1005.723011,1947,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V130000001S70011963510113,13,1,NaN,2,432.646602,840.657411,1968,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Metro Evictions, Metro Behind Rent

In [42]:
metro_evict = df1[df1.EVICT>0]
metro_evict['evict_bool'] = metro_evict.apply(lambda x: 1 if x.EVICT<3 else 0,axis=1)
metro_evict_ct = full_crosstab(metro_evict[metro_evict.metro!='remove'],
                         ['WEEK','metro', 'evict_bool'],
                         'PWEIGHT',
                         ['WEEK','metro'],
                         critical_val=1).round(2)
metro_evict_ct[(metro_evict_ct.evict_bool==1)&(metro_evict_ct.metro=='Phoenix')]

,WEEK,metro,evict_bool,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions
23,13,Phoenix,1,49933.31,18323.58,18323.58,102490.28,26782.72,26782.72,0.49
53,14,Phoenix,1,22140.62,6142.48,6142.48,68365.49,17708.66,17708.66,0.32
83,15,Phoenix,1,28129.84,9734.82,9734.82,83649.02,22477.15,22477.15,0.34
113,16,Phoenix,1,24733.99,8595.58,8595.58,94287.97,24035.15,24035.15,0.26
143,17,Phoenix,1,51226.65,12256.61,12256.61,100013.57,23849.52,23849.52,0.51
173,18,Phoenix,1,161165.39,68020.23,68020.23,220968.36,71308.08,71308.08,0.73
203,19,Phoenix,1,46668.35,14470.31,14470.31,112898.81,31522.69,31522.69,0.41
233,20,Phoenix,1,68250.37,21987.93,21987.93,122071.86,28173.88,28173.88,0.56
263,21,Phoenix,1,44242.74,16934.26,16934.26,106262.11,26537.86,26537.86,0.42
293,22,Phoenix,1,84811.47,32687.70,32687.70,109704.60,36972.23,36972.23,0.77


In [39]:
#add occupied without rent and include with behind? people much more worried about eviction than ppl behind on rent
metro_rent_ct = full_crosstab(df1[-(df1.rent=='remove')],
                         ['WEEK','metro', 'rent'],
                         'PWEIGHT',
                         ['WEEK','metro'],
                         critical_val=1).round(2)
metro_rent_ct[(metro_rent_ct.rent=='behind')&(metro_rent_ct.metro=='Phoenix')]

,WEEK,metro,rent,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions
22,13,Phoenix,behind,102490.28,26782.72,26782.72,737756.09,61180.29,61180.29,0.14
54,14,Phoenix,behind,68365.49,17708.66,17708.66,806460.14,53276.17,53276.17,0.08
86,15,Phoenix,behind,97878.93,27011.71,27011.71,898643.14,59301.43,59301.43,0.11
118,16,Phoenix,behind,94287.97,24035.15,24035.15,786646.34,67281.99,67281.99,0.12
150,17,Phoenix,behind,100013.57,23849.52,23849.52,992940.79,85689.57,85689.57,0.10
182,18,Phoenix,behind,220968.36,71308.08,71308.08,879308.05,107214.54,107214.54,0.25
214,19,Phoenix,behind,115121.44,31379.48,31379.48,816134.71,66145.36,66145.36,0.14
246,20,Phoenix,behind,136012.60,30094.51,30094.51,731107.29,61934.40,61934.40,0.19
278,21,Phoenix,behind,106262.11,26537.86,26537.86,741455.52,71069.38,71069.38,0.14
310,22,Phoenix,behind,109704.60,36972.23,36972.23,750766.92,71089.52,71089.52,0.15


In [200]:
#full_crosstab(df, ['WEEK','metro'], 'PWEIGHT', ['WEEK','metro'], critical_val=1.645).round(2)
df1 = df[df.WEEK==13].copy()
df1 = df1[df1.metro=='Atlanta']
full_crosstab(df1, ['metro','race','rent'], 'PWEIGHT', ['metro','race'], critical_val=1.645).round(2)

,metro,race,rent,PWEIGHT,std_err,mrgn_err,proportions
0,Atlanta,Asian,behind,5923.87,3624.72,5962.66,0.22
1,Atlanta,Asian,current,20534.83,7041.32,11582.98,0.78
2,Atlanta,Other,behind,8375.40,5765.38,9484.05,0.31
3,Atlanta,Other,current,18352.92,6439.09,10592.30,0.69
4,Atlanta,black,behind,84377.39,25286.83,41596.84,0.20
5,Atlanta,black,current,334560.43,52043.74,85611.95,0.80
6,Atlanta,white,behind,32461.67,13918.24,22895.50,0.08
7,Atlanta,white,current,361599.05,46427.68,76373.54,0.92


In [217]:
detail = full_crosstab(df1, ['WEEK','state','race','rent'], 'PWEIGHT', ['WEEK','state','race'], critical_val=1.645).round(2)
top = full_crosstab(df1, ['WEEK','state','race'], 'PWEIGHT', ['WEEK','state'], critical_val=1.645).round(2)
rv = detail.merge(top,'left',['WEEK','state','race'],suffixes=('_full','_demo'))
y = rv[rv.rent=='behind']
y_2 = y[y.proportions_full>0.4]
y_3 = y_2[y_2.PWEIGHT_full>y_2.mrgn_err_full]
y_3.state.unique()


array(['Alabama', 'Iowa', 'North Carolina', 'South Dakota', 'Virginia',
       'Illinois', 'Minnesota', 'New York', 'Ohio', 'Washington',
       'Connecticut', 'Indiana', 'New Mexico', 'Oregon', 'Georgia',
       'Louisiana', 'Maryland', 'Michigan', 'Pennsylvania', 'Wisconsin',
       'Alaska', 'District of Columbia', 'Nevada', 'Tennessee',
       'Mississippi', 'Nebraska', 'Oklahoma', 'Rhode Island',
       'South Carolina', 'Missouri', 'Texas', 'Colorado', 'Delaware',
       'Florida', 'Kentucky', 'Massachusetts', 'Vermont', 'California',
       'Kansas', 'New Jersey', 'Utah'], dtype=object)

In [209]:
detail_r = full_crosstab(df1, ['WEEK','region','race','rent'], 'PWEIGHT', ['WEEK','region','race'], critical_val=1.645).round(2)
top_r = full_crosstab(df1, ['WEEK','region','race'], 'PWEIGHT', ['WEEK','region'], critical_val=1.645).round(2)
rv_r = detail_r.merge(top_r,'left',['WEEK','region','race'],suffixes=('_full','_demo'))
x = rv_r[rv_r.rent=='behind']
x[x.proportions_full>0.4]

,WEEK,region,race,rent,PWEIGHT_full,std_err_full,mrgn_err_full,proportions_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions_demo
172,18,NE,black,behind,857991.48,189038.01,310967.52,0.44,1928187.96,195373.94,321390.14,0.20
292,22,MW,black,behind,733948.69,156260.04,257047.76,0.50,1479393.27,156888.39,258081.41,0.17
298,22,NE,Other,behind,396776.44,110644.16,182009.64,0.57,701300.10,117815.62,193806.69,0.08


In [22]:
check_1 = pd.read_csv('../data/puf_housing_data_remapped_labels.csv')

## BULK CROSSTABS

In [159]:
SELECT_ALL_QUESTIONS = ['SPNDSRC1', 'SPNDSRC2', 'SPNDSRC3', 'SPNDSRC4', 'SPNDSRC5', 'SPNDSRC6', 'SPNDSRC7', 'SPNDSRC8']
index_list = ['EST_MSA', 'WEEK']
crosstab_list = ['RRACE', 'EEDUC', 'INCOME']
question_list = ['SPNDSRC1', 'SPNDSRC2', 'SPNDSRC3', 'SPNDSRC4', 'SPNDSRC5', 'SPNDSRC6', 'SPNDSRC7', 'SPNDSRC8', 'RENTCUR', 'MORTCUR', 'MORTCONF', 'EVICT', 'FORCLOSE']
bulk_crosstabs(check_1, index_list, crosstab_list, question_list, weight='PWEIGHT', critical_val=1)
# idx one at a time? level of proportions? TODO: Fix proportion calc w/ NA
# -99 is DNR

,EST_MSA,WEEK,ct_val,q_val,PWEIGHT_full,std_err_full,mrgn_err_full,PWEIGHT_demo,std_err_demo,mrgn_err_demo,proportions,ct_var,q_var
0,12060.0,13,1 - White,0 - not selected,550241.01,65737.99,65737.99,2263934.59,81694.78,81694.78,0.24,RRACE,SPNDSRC1
1,12060.0,13,1 - White,1 - Regular income sources,1713693.58,65061.57,65061.57,2263934.59,81694.78,81694.78,0.76,RRACE,SPNDSRC1
2,12060.0,13,2 - Black,0 - not selected,693102.10,65196.48,65196.48,1413499.92,67904.84,67904.84,0.49,RRACE,SPNDSRC1
3,12060.0,13,2 - Black,1 - Regular income sources,720397.82,63839.39,63839.39,1413499.92,67904.84,67904.84,0.51,RRACE,SPNDSRC1
4,12060.0,13,3 - Asian,0 - not selected,66136.81,17408.49,17408.49,176090.67,26180.63,26180.63,0.38,RRACE,SPNDSRC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,47900.0,22,"5 - $75,000 - $99,999",2 - Not very or not at all likely to leave thi...,16516.31,9044.43,9044.43,17980.58,9202.89,9202.89,0.92,INCOME,FORCLOSE
1822,47900.0,22,"6 - $100,000 - $149,999",1 - Somewhat to very likely to leave this home...,2059.88,2109.63,2109.63,25858.64,8915.82,8915.82,0.08,INCOME,FORCLOSE
1823,47900.0,22,"6 - $100,000 - $149,999",2 - Not very or not at all likely to leave thi...,23798.76,8649.52,8649.52,25858.64,8915.82,8915.82,0.92,INCOME,FORCLOSE
1824,47900.0,22,"7 - $150,000 - $199,999",2 - Not very or not at all likely to leave thi...,4492.10,3688.95,3688.95,4492.10,3688.95,3688.95,1.00,INCOME,FORCLOSE


In [150]:
x = [1,2,3]
x +['h']

[1, 2, 3, 'h']

In [157]:
check_1.columns

Index(['Unnamed: 0', 'SCRAM', 'WEEK', 'EST_ST', 'EST_MSA', 'REGION', 'HWEIGHT',
       'PWEIGHT', 'TBIRTH_YEAR', 'ABIRTH_YEAR',
       ...
       'PWEIGHT72', 'PWEIGHT73', 'PWEIGHT74', 'PWEIGHT75', 'PWEIGHT76',
       'PWEIGHT77', 'PWEIGHT78', 'PWEIGHT79', 'PWEIGHT80', 'topline'],
      dtype='object', length=331)